In [1]:
import json
from concurrent.futures import ThreadPoolExecutor

import boto3
from botocore import UNSIGNED
from botocore.config import Config
import pandas as pd
from web3 import Web3

from mainnet_launch.database.schema.full import RebalancePlans, Destinations, DexSwapSteps, Tokens

from mainnet_launch.database.schema.postgres_operations import (
    get_full_table_as_orm,
    get_full_table_as_df,
    insert_avoid_conflicts,
    get_subset_not_already_in_column,
)

from mainnet_launch.constants import ALL_AUTOPOOLS, AutopoolConstants, USDC, WETH, AUTO_ETH


autopool = AUTO_ETH


s3_client = boto3.client("s3", config=Config(signature_version=UNSIGNED))
plans: list[RebalancePlans] = get_full_table_as_df(
    RebalancePlans, where_clause=RebalancePlans.chain_id == autopool.chain.chain_id
    )
plans

2025-05-27 12:57:13.692 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


2025-05-27 12:57:14,637 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-05-27 12:57:14,637 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-27 12:57:14,837 INFO sqlalchemy.engine.Engine select current_schema()
2025-05-27 12:57:14,837 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-27 12:57:15,028 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-05-27 12:57:15,028 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-27 12:57:15,216 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-27 12:57:15,217 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

,file_name,datetime_generated,autopool_vault_address,chain_id,solver_address,rebalance_type,destination_out,token_out,destination_in,token_in,...,in_dest_apr,in_dest_adj_apr,apr_delta,swap_offset_period,num_candidate_destinations,candidate_destinations_rank,projected_swap_cost,projected_net_gain,projected_gross_gain,projected_slippage
0,rebalance_plan_1727897421_0x0A2b94F6871c1D7A32...,2024-10-02 19:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0x896eCc16Ab4AFfF6cE0765A5B924BaECd7Fa455a,0xe080027Bd47353b5D1639772b4a75E9Ed3658A0d,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,0.000000,0.092285,0.092285,50.0,12.0,0.0,1.989009e-01,1.003935,1.202836,0.199381
1,rebalance_plan_1736580621_0x0A2b94F6871c1D7A32...,2025-01-11 07:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0x88794C65550DeB6b4087B7552eCf295113794410,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,0.000000,0.086773,0.086773,46.0,5.0,0.0,5.690695e-02,1.388123,1.445030,0.041353
2,rebalance_plan_1739302221_0x0A2b94F6871c1D7A32...,2025-02-11 19:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0xbA1462f43c6f60ebD1C62735c94E428aD073E01A,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,0.000000,0.089842,0.089842,42.0,4.0,0.0,2.562382e-02,0.408126,0.433750,0.059284
3,rebalance_plan_1737682221_0x0A2b94F6871c1D7A32...,2025-01-24 01:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rebalance_plan_1743481821_0x0A2b94F6871c1D7A32...,2025-04-01 04:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3383,rebalance_plan_1748049286_0xa7569A44f348d3D70d...,2025-05-24 01:14:46+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,Pool2Pool,0x7876F91BB22148345b3De16af9448081E9853830,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,0x7583b1589aDD33320366A48A92794D77763FAE9e,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,...,0.057203,0.079587,0.022384,14.0,3.0,0.0,3.237657e-11,86.053494,86.053494,0.032362
3384,rebalance_plan_1748117686_0xa7569A44f348d3D70d...,2025-05-24 20:14:46+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,Pool2Pool,0x7876F91BB22148345b3De16af9448081E9853830,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,0x7583b1589aDD33320366A48A92794D77763FAE9e,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,...,0.059091,0.077252,0.018161,14.0,3.0,0.0,3.580927e-11,69.749488,69.749488,0.035797
3385,rebalance_plan_1748340886_0xa7569A44f348d3D70d...,2025-05-27 10:14:46+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,Pool2Pool,0x7876F91BB22148345b3De16af9448081E9853830,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,...,0.059546,0.066662,0.007116,14.0,3.0,0.0,3.003755e-11,17.705725,17.705725,0.030034
3386,rebalance_plan_1748031286_0xa7569A44f348d3D70d...,2025-05-23 20:14:46+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0x7876F91BB22148345b3De16af9448081E9853830,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,...,0.000000,0.078102,0.078102,14.0,3.0,0.0,7.613385e-12,137.985141,137.985141,0.015000


In [3]:
from mainnet_launch.data_fetching.tokemak_subgraph import fetch_autopool_rebalance_events_from_subgraph

rebalance_event_df = fetch_autopool_rebalance_events_from_subgraph(autopool)

rebalance_event_df

ValueError: ('bad chain', ChainData(name='eth', block_autopool_first_deployed=20752910, approx_seconds_per_block=12.0, chain_id=1, start_unix_timestamp=1726365887))